In [19]:
import os

from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from torchvision import transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, transform
import torch
from torchvision.utils import make_grid
from tqdm import tqdm

In [43]:
from datasets import load_dataset
dataset = load_dataset("./data/labels",split="train")
dogs = dataset.train_test_split(test_size=0.2)

In [44]:
breed = "data/breed.csv"
filename = "data/labels.csv"
labelsNames = ["id", "breed"]
csvlabels = read_csv(filename, names=labelsNames)

breedNames = read_csv(breed)["breed"].tolist()[1:]
print(csvlabels.shape)
# print(len(breedNames))

(12001, 2)


In [46]:
labels = csvlabels["breed"]
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
labels

0                      breed
1                  chihuahua
2                  chihuahua
3                  chihuahua
4                  chihuahua
                ...         
11996    african_hunting_dog
11997    african_hunting_dog
11998    african_hunting_dog
11999    african_hunting_dog
12000    african_hunting_dog
Name: breed, Length: 12001, dtype: object

In [5]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [6]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [7]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

<generator object HfApi.list_datasets at 0x0000027E636C09E0>

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer
checkpoint = "google/vit-base-patch16-224-in21k"

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    label2id=train_loader.dataset.data_label,
    id2label={v: k for k, v in train_loader.dataset.data_label.items()},
)